In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')
import requests

from bs4 import BeautifulSoup
import lxml.html as lh

Libraries imported.


Let's use BeautifulShop library to read table from HTML web page code. First  i check if i'm reading the table correctly and than reading every column into array

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)

doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [6]:
#Checking number of columns - shouuld be 3 like n the table
[len(T) for T in tr_elements[0:1]]

[3]

In [15]:
#Checking Name of the columns
col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))
    
len(tr_elements)
len(col)

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


3

In [16]:
#Read text information into columns

for j in range(1,181):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [21]:
#Checking number of the rows
[len(C) for (title,C) in col]

[180, 180, 180]

In [23]:
#Converting into Pandas
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()


,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Now i need to clean garbage. First rename columns, second remove "Not Assigned" Boroughs and than remove unnecessary 

In [24]:
df.columns = ['Postal Code','Borough','Neighborhood']
df['Postal Code']=df['Postal Code'].str.replace('\n', '')
df['Borough']=df['Borough'].str.replace('\n', '')
df['Neighborhood']=df['Neighborhood'].str.replace('\n', '')

df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df = df.reset_index()
df.drop(['index'], axis=1,inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [25]:
df.shape

(103, 3)